# About this notebook
- Deberta-v3 xsmall huggingface version
- Previous notebook is [here](https://www.kaggle.com/code/piantic/train-lmsys-deberta-v3-starter-code/notebook)
- This notebook introduces the use of three texts. - `prompt`, `res_a`, `res_b`
- Inference notebook will be released at a later date.


If this notebook is helpful, feel free to upvote.

And please upvote the original notebook :)

`V1` - GPU T4x2 and initial settings

`V2` - only use `prompt` because of problem where only 1 label appears during prediction

`V3` - add truncate function for texts

In [1]:
import os
os.environ["CUDA_VISIBLE_DEVICES"]="0,1"
import re
import random
import warnings
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import torch

os.system('python -m pip install --no-index --find-links=../input/lmsys-pip-wheels transformers')
os.system('python -m pip install --no-index --find-links=../input/lmsys-pip-wheels tokenizers')

from transformers import AutoTokenizer, AutoModelForSequenceClassification
from transformers import TrainingArguments, Trainer
from transformers import DataCollatorWithPadding
from datasets import Dataset
from sklearn.model_selection import train_test_split
from sklearn.model_selection import StratifiedKFold
from sklearn.metrics import confusion_matrix, ConfusionMatrixDisplay
from sklearn.metrics import log_loss
from tokenizers import AddedToken
warnings.simplefilter('ignore')

Looking in links: ../input/lmsys-pip-wheels
Looking in links: ../input/lmsys-pip-wheels


2024-06-16 02:56:45.049339: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:9261] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2024-06-16 02:56:45.049471: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:607] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2024-06-16 02:56:45.220631: E external/local_xla/xla/stream_executor/cuda/cuda_blas.cc:1515] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered


In [2]:
# ====================================================
# Directory settings
# ====================================================
OUTPUT_DIR = './'
if not os.path.exists(OUTPUT_DIR):
    os.makedirs(OUTPUT_DIR)

In [3]:
train = pd.read_csv('/kaggle/input/lmsys-chatbot-arena/train.csv')
test = pd.read_csv('/kaggle/input/lmsys-chatbot-arena/test.csv')
submission = pd.read_csv('/kaggle/input/lmsys-chatbot-arena/sample_submission.csv')

In [4]:
train

,id,model_a,model_b,prompt,response_a,response_b,winner_model_a,winner_model_b,winner_tie
0,30192,gpt-4-1106-preview,gpt-4-0613,"[""Is it morally right to try to have a certain...","[""The question of whether it is morally right ...","[""As an AI, I don't have personal beliefs or o...",1,0,0
1,53567,koala-13b,gpt-4-0613,"[""What is the difference between marriage lice...","[""A marriage license is a legal document that ...","[""A marriage license and a marriage certificat...",0,1,0
2,65089,gpt-3.5-turbo-0613,mistral-medium,"[""explain function calling. how would you call...","[""Function calling is the process of invoking ...","[""Function calling is the process of invoking ...",0,0,1
3,96401,llama-2-13b-chat,mistral-7b-instruct,"[""How can I create a test set for a very rare ...","[""Creating a test set for a very rare category...","[""When building a classifier for a very rare c...",1,0,0
4,198779,koala-13b,gpt-3.5-turbo-0314,"[""What is the best way to travel from Tel-Aviv...","[""The best way to travel from Tel Aviv to Jeru...","[""The best way to travel from Tel-Aviv to Jeru...",0,1,0
...,...,...,...,...,...,...,...,...,...
57472,4294656694,gpt-4-0613,claude-1,"[""A simple mnemonic for \u03c0:\n\""How I wish ...","[""Sure, let's break it down:\n\n1. \""How\"" has...","[""Here is how that mnemonic represents the dig...",1,0,0
57473,4294692063,claude-2.0,llama-2-13b-chat,"[""In python, implement a naive Bayes with gaus...","[""Here is an implementation of a naive Bayes c...","[""Sure! Here's an implementation of a naive Ba...",1,0,0
57474,4294710549,claude-1,alpaca-13b,"[""is it unethical to work on building weapons?...","[""Working on weapons technology raises some et...","[""It depends on the context. Weapons can be us...",1,0,0
57475,4294899228,palm-2,tulu-2-dpo-70b,"[""If a bait contains 0,0025% bromadiolon then ...","[""Bromadiolone is a rodenticide which is most ...","[""As an AI language model, I do not promote or...",0,1,0


In [5]:
class CFG:
    n_splits = 5
    seed = 42
    max_length = 1539 # 512 x 3 + a
    lr = 1e-5
    train_batch_size = 8
    eval_batch_size = 4
    train_epochs = 4
    weight_decay = 0.01
    warmup_ratio = 0.1
    num_labels = 3
    debug=False
    model = "microsoft/deberta-v3-xsmall"
    target_cols = ['winner_model_a', 'winner_model_b', 'winner_tie']

# Add Features

In [6]:
# https://www.kaggle.com/code/piantic/train-lmsys-deberta-v3-starter-code/notebook

def add_label(df):
    labels = np.zeros(len(df), dtype=np.int32)
    labels[df['winner_model_a'] == 1] = 0
    labels[df['winner_model_b'] == 1] = 1
    labels[df['winner_tie'] == 1] = 2
    df['label'] = labels
    return df


def add_stats(df):
    # Some stats
    df["prompt_words"] = df["prompt"].apply(lambda x: x.replace('\n', ' ').split(" "))
    df["total_prompt_words"] = df["prompt"].apply(lambda x: len(x.split(" ")))
    df["prompt_length"] = df["prompt"].apply(lambda x: len(x))

    df["response_a_words"] = df["response_a"].apply(lambda x: x.replace('\n', ' ').split(" "))
    df["total_response_a_words"] = df["response_a"].apply(lambda x: len(x.split(" ")))
    df["response_a_length"] = df["response_a"].apply(lambda x: len(x))

    df["response_b_words"] = df["response_b"].apply(lambda x: x.replace('\n', ' ').split(" "))
    df["total_response_b_words"] = df["response_b"].apply(lambda x: len(x.split(" ")))
    df["response_b_length"] = df["response_b"].apply(lambda x: len(x))
    
    return df

def truncate_text(df, column_name, max_length=1000000):
    df[f"{column_name}"] = df[column_name].str[:max_length]
    return df

train = add_label(train)
train = add_stats(train)
train = truncate_text(train, 'prompt')
train = truncate_text(train, 'response_a')
train = truncate_text(train, 'response_b')

# Tokenization
- In this notebook, i will use `prompt`, `response_a`, `response_b`.

In [14]:
class Tokenize(object):
    def __init__(self, train, valid):
        self.tokenizer = AutoTokenizer.from_pretrained(CFG.model)
        self.train = train
        self.valid = valid
        
    def get_dataset(self, df):
        ds = Dataset.from_dict({""
                'id': [e for e in df['id']],
                'prompt': [ft for ft in df['prompt']],
                'response_a': [ft for ft in df['response_a']],
                'response_b': [ft for ft in df['response_b']],
                'labels': [s for s in df['label']],
            })
        return ds
    
    def tokenize_function(self, df):
        tokenized_inputs = self.tokenizer(
            df['prompt'], df['response_a'] + self.tokenizer.sep_token + df['response_b'],
            truncation=True, padding=True, max_length=CFG.max_length
        )
        return tokenized_inputs
    
    def __call__(self):
        train_ds = self.get_dataset(train)
        valid_ds = self.get_dataset(valid)
        
        tokenized_train = train_ds.map(
            self.tokenize_function
        )
        tokenized_valid = valid_ds.map(
            self.tokenize_function
        )
        
        return tokenized_train, tokenized_valid, self.tokenizer

# Compute metrics

In [8]:
def compute_metrics(eval_pred):
    predictions, labels = eval_pred
    score = log_loss(labels, predictions)
    results = {
        'score': score
    }
    return results

# CV split

In [10]:
data = train.copy()
data["label"] = data["label"].astype('int32')
skf = StratifiedKFold(n_splits=CFG.n_splits, shuffle=True, random_state=CFG.seed)
for i, (_, val_index) in enumerate(skf.split(data, data["label"])):
    data.loc[val_index, "fold"] = i
data.head(5)

,id,model_a,model_b,prompt,response_a,response_b,winner_model_a,winner_model_b,winner_tie,label,prompt_words,total_prompt_words,prompt_length,response_a_words,total_response_a_words,response_a_length,response_b_words,total_response_b_words,response_b_length,fold
0,30192,gpt-4-1106-preview,gpt-4-0613,"[""Is it morally right to try to have a certain...","[""The question of whether it is morally right ...","[""As an AI, I don't have personal beliefs or o...",1,0,0,0,"[[""Is, it, morally, right, to, try, to, have, ...",28,165,"[[""The, question, of, whether, it, is, morally...",656,4538,"[[""As, an, AI,, I, don't, have, personal, beli...",204,1206,4.0
1,53567,koala-13b,gpt-4-0613,"[""What is the difference between marriage lice...","[""A marriage license is a legal document that ...","[""A marriage license and a marriage certificat...",0,1,0,1,"[[""What, is, the, difference, between, marriag...",35,200,"[[""A, marriage, license, is, a, legal, documen...",537,3114,"[[""A, marriage, license, and, a, marriage, cer...",591,3649,3.0
2,65089,gpt-3.5-turbo-0613,mistral-medium,"[""explain function calling. how would you call...","[""Function calling is the process of invoking ...","[""Function calling is the process of invoking ...",0,0,1,2,"[[""explain, function, calling., how, would, yo...",9,60,"[[""Function, calling, is, the, process, of, in...",141,921,"[[""Function, calling, is, the, process, of, in...",282,1835,3.0
3,96401,llama-2-13b-chat,mistral-7b-instruct,"[""How can I create a test set for a very rare ...","[""Creating a test set for a very rare category...","[""When building a classifier for a very rare c...",1,0,0,0,"[[""How, can, I, create, a, test, set, for, a, ...",18,87,"[[""Creating, a, test, set, for, a, very, rare,...",536,3182,"[[""When, building, a, classifier, for, a, very...",265,1562,0.0
4,198779,koala-13b,gpt-3.5-turbo-0314,"[""What is the best way to travel from Tel-Aviv...","[""The best way to travel from Tel Aviv to Jeru...","[""The best way to travel from Tel-Aviv to Jeru...",0,1,0,1,"[[""What, is, the, best, way, to, travel, from,...",14,79,"[[""The, best, way, to, travel, from, Tel, Aviv...",236,1300,"[[""The, best, way, to, travel, from, Tel-Aviv,...",122,772,3.0


In [11]:
if CFG.debug:
    display(data.groupby('fold').size())
    data = data.sample(n=1000, random_state=0).reset_index(drop=True)
    display(data.groupby('fold').size())

# Set Training Args

In [12]:
training_args = TrainingArguments(
    output_dir=OUTPUT_DIR,
    fp16=True,
    learning_rate=CFG.lr,
    per_device_train_batch_size=CFG.train_batch_size,
    per_device_eval_batch_size=CFG.eval_batch_size,
    num_train_epochs=CFG.train_epochs,
    weight_decay=CFG.weight_decay,
    evaluation_strategy='epoch',
    metric_for_best_model='score',
    save_strategy='epoch',
    save_total_limit=1,
    load_best_model_at_end=True,
    report_to='none',
    warmup_ratio=CFG.warmup_ratio,
    optim='adamw_torch'
)

# Training by hold

In [15]:
for fold in range(len(data['fold'].unique())):
    train = data[data['fold'] != fold]
    valid = data[data['fold'] == fold]
    
    # ADD NEW TOKENS for ("\n") new paragraph and (" "*2) double space
    tokenizer = AutoTokenizer.from_pretrained(CFG.model)
    tokenizer.add_tokens([AddedToken("\n", normalized=False)])
    tokenizer.add_tokens([AddedToken(" "*2, normalized=False)])
    tokenize = Tokenize(train, valid)
    tokenized_train, tokenized_valid, tokenizer = tokenize()
    
    model = AutoModelForSequenceClassification.from_pretrained(CFG.model, num_labels=CFG.num_labels)
    data_collator = DataCollatorWithPadding(tokenizer=tokenizer)
    
    trainer = Trainer(
        model=model,
        args=training_args,
        train_dataset=tokenized_train,
        eval_dataset=tokenized_valid,
        data_collator=data_collator,
        tokenizer=tokenizer,
        compute_metrics=compute_metrics
    )
    
    trainer.train()
    
    y_true = valid['label'].values
    predictions = trainer.predict(tokenized_valid).predictions
    predictions = predictions.argmax(axis=1)
    cm = confusion_matrix(y_true, predictions, labels=[x for x in range(0,3)])
    draw_cm = ConfusionMatrixDisplay(confusion_matrix=cm,
                                  display_labels=[x for x in range(0,3)])
    draw_cm.plot()
    plt.show()
    
    trainer.save_model(f'deberta-v3-xsmall_fold_{fold}')
    tokenizer.save_pretrained(f'deberta-v3-xsmall_fold_{fold}')
    
    valid.to_csv(f'valid_df_fold_{fold}.csv', index=False)
    break # just fold 0 train for test

Map:   0%|          | 0/45981 [00:00<?, ? examples/s]

Map:   0%|          | 0/11496 [00:00<?, ? examples/s]

pytorch_model.bin:   0%|          | 0.00/241M [00:00<?, ?B/s]

Some weights of DebertaV2ForSequenceClassification were not initialized from the model checkpoint at microsoft/deberta-v3-xsmall and are newly initialized: ['classifier.bias', 'classifier.weight', 'pooler.dense.bias', 'pooler.dense.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


OutOfMemoryError: Caught OutOfMemoryError in replica 0 on device 0.
Original Traceback (most recent call last):
  File "/opt/conda/lib/python3.10/site-packages/torch/nn/parallel/parallel_apply.py", line 85, in _worker
    output = module(*input, **kwargs)
  File "/opt/conda/lib/python3.10/site-packages/torch/nn/modules/module.py", line 1518, in _wrapped_call_impl
    return self._call_impl(*args, **kwargs)
  File "/opt/conda/lib/python3.10/site-packages/torch/nn/modules/module.py", line 1527, in _call_impl
    return forward_call(*args, **kwargs)
  File "/opt/conda/lib/python3.10/site-packages/transformers/models/deberta_v2/modeling_deberta_v2.py", line 1300, in forward
    outputs = self.deberta(
  File "/opt/conda/lib/python3.10/site-packages/torch/nn/modules/module.py", line 1518, in _wrapped_call_impl
    return self._call_impl(*args, **kwargs)
  File "/opt/conda/lib/python3.10/site-packages/torch/nn/modules/module.py", line 1527, in _call_impl
    return forward_call(*args, **kwar

In [16]:
tokenized_train[0]

{'id': 30192,
 'prompt': '["Is it morally right to try to have a certain percentage of females on managerial positions?","OK, does pineapple belong on a pizza? Relax and give me fun answer."]',
 'response_a': '["The question of whether it is morally right to aim for a certain percentage of females in managerial positions is a complex ethical issue that involves considerations of fairness, equality, diversity, and discrimination.\\n\\nHere are some arguments in favor of and against such policies:\\n\\n**Arguments in favor:**\\n\\n1. **Correcting Historical Inequities:** Women have historically been underrepresented in leadership roles due to various cultural, institutional, and social barriers. Aiming for a specific percentage can be seen as a corrective measure to address past and ongoing discrimination.\\n\\n2. **Promoting Diversity:** Diverse leadership teams can enhance decision-making and represent a broader range of perspectives. This can lead to better outcomes for organizations 

# References
- https://www.kaggle.com/code/hashidoyuto/deberta-baseline-aes2-0-train
- https://www.kaggle.com/code/cdeotte/deberta-v3-small-starter-cv-0-820-lb-0-800?scriptVersionId=174239814